In [ ]:
cd drive/MyDrive/JAIST/Research/RL/movie-lens-simulator

In [ ]:
!make style
!make setup

In [ ]:
from src.data.movielens1M import LoadMovielens1M, PreprocessMovielens1M
from src.dataloader.movielens import MovieLensDataset
from src.dataloader.dataloader import get_MovieLensDataloaders
from src.model.embedding import Simulator
from src.trainer.rl_trainer import rl_trainer
from src.trainer.embedding_trainer import Embedding_Trainer
from src.recommend.recommend_handler import RecommendHandler
from src.evaluation.precision_oriented import PrecisionOrientedMetrics
from src.evaluation.diversity_oriented import DiversityOrientedMetrics

from configs.config import CFG_DICT
from src.utils.file_handler import File_Handler

from src.agent.actor_critic import ActorCriticAgent

import numpy as np
import torch
import pandas as pd

file_handler = File_Handler()
logger = file_handler.make_logger_file()

In [ ]:
load_rawML1M = LoadMovielens1M(path=CFG_DICT['DATASET']['PATH'])
raw_datas = load_rawML1M.load_data()

preprocessML1M = PreprocessMovielens1M(raw_datas)
preprocess_datas = preprocessML1M.apply_prepocess()

ratings, users, movies, histories = preprocess_datas

In [ ]:
preprocess_datas = ratings[ratings.UserID < 100], users, movies, histories

device = 'cuda'

recommend_handler = RecommendHandler(
    initial_data = preprocess_datas,
    topK = CFG_DICT['SIMULATION']['topK'],
    device = device,
    file_handler = file_handler,
    )

user_simulator = Simulator(emb_size=CFG_DICT['USER_SIMULATOR']['DIM_EMB']).to(device)
initial_recommender = Simulator(emb_size=CFG_DICT['INITIAL_RECOMMENDER']['DIM_EMB']).to(device)

agent = ActorCriticAgent(device=device)

log_metrics = pd.DataFrame()


for time in np.arange(CFG_DICT['SIMULATION']['EPOCH']):

    ratings, _, _, _ = preprocess_datas

    logger.info('-'*50)
    logger.info('time : {}'.format(time))
    logger.info('data len : {}'.format(len(ratings)))

    if time != 0:
        user_simulator.load_state_dict(file_handler.load('user_simulator_0.pkl'))

    ### user
    user_simlator_datas = get_MovieLensDataloaders(
        model_type = 'USER_SIMULATOR',
        processed_data = preprocess_datas
    )

    user_simulator_trainer = Embedding_Trainer(
        model_type = 'USER_SIMULATOR',
        model = user_simulator,
        loaders = user_simlator_datas,
        device = device
        )

    user_simulator = user_simulator_trainer.train()
    file_handler.save(user_simulator.state_dict(), 'user_simulator_{}.pkl'.format(time))


    ### recommend
    if time == 0:
        initial_recommender_datas = get_MovieLensDataloaders(
            model_type = 'INITIAL_RECOMMENDER',
            processed_data = preprocess_datas
        )

        initial_recommender_trainer = Embedding_Trainer(
            model_type = 'INITIAL_RECOMMENDER',
            model = initial_recommender,
            loaders = initial_recommender_datas,
            device = device
            )

        initial_recommender = initial_recommender_trainer.train()
        file_handler.save(initial_recommender.state_dict(), 'initial_recommender_{}.pkl'.format(time))

        recommend_handler.register_models((initial_recommender, user_simulator))
        recommend_handler.recommend(time=time)
    
    else:

        if len(recommend_handler.buffer) > CFG_DICT['REPLAY_BUFFER']['BS']:
            bs = CFG_DICT['REPLAY_BUFFER']['BS']
        else:
            bs = int(len(recommend_handler.buffer) / 2)

        agent, recommend_handler = rl_trainer(
            recommend_handler = recommend_handler,
            agent = agent,
            device = device,
            bs = bs
        )

        # 学習
        for step in np.arange(CFG_DICT['RL']['EPOCH']):
            state, action, reward, next_state, prob_behavior = recommend_handler.buffer.get_batch(bs)
            prob_target = agent.get_target_probs(state.to(device), action.to(torch.long))
            agent.update(state.to(device), reward.to(device), next_state.to(device), prob_behavior.to(device), prob_target.to(device))

        recommend_handler.register_models((agent, user_simulator))
        recommend_handler.recommend(time=time)

    # evaluation
    pos = PrecisionOrientedMetrics(recommend_handler.pred_ratings)
    MAP = pos.map() ; NDCG = pos.ndcg(); MRR = pos.mrr(); HR = pos.hr()

    dom = DiversityOrientedMetrics(recommend_handler.pred_ratings, recommend_handler.ratings, topK=CFG_DICT['SIMULATION']['topK'])
    DIVERSITY = dom.diversity_score(); SERENDIPITY = dom.serendipity_score() ; NOVELTY = dom.novelty_score(); UNIQUENESS = dom.uniqueness_score()

    logger.info("MAP : {}".format(MAP))
    logger.info("NDCG : {}".format(NDCG))
    logger.info("MRR : {}".format(MRR))
    logger.info("HR : {}".format(HR))
    logger.info("DIVERSITY : {}".format(DIVERSITY))
    logger.info("SERENDIPITY : {}".format(SERENDIPITY))
    logger.info("NOVELTY : {}".format(NOVELTY))
    logger.info("UNIQUENESS : {}".format(UNIQUENESS))

    log_metrics = log_metrics.append({
        'MAP': MAP, 
        'NDCG': NDCG, 
        'MRR': MRR,
        'HR' : HR,
        'DIVERSITY': DIVERSITY, 
        'SERENDIPITY': SERENDIPITY, 
        'NOVELTY': NOVELTY,
        'UNIQUENESS' : UNIQUENESS,
        }, ignore_index=True)
    
    file_handler.save(log_metrics, 'log_metrics.csv')

    # update ratings
    recommend_handler.update_ratings()
    del preprocess_datas

    preprocess_datas = (recommend_handler.ratings, users, movies, recommend_handler.history)